# DB Connection Smoke TestsEnvironment variables are provided by docker-compose via `.env`.

In [ ]:
import os, sys, platformprint('Python:', sys.version)print('Platform:', platform.platform())env_keys = [k for k in os.environ.keys() if k.startswith(('MSSQL_','MARIADB_'))]print('Loaded env keys:', sorted(env_keys))

## Test SQL Server (pyodbc via FreeTDS)

In [ ]:
import os, pyodbcm_host = os.getenv('MSSQL_HOST', 'host.docker.internal')m_port = os.getenv('MSSQL_PORT', '1433')m_db   = os.getenv('MSSQL_DB', '')m_user = os.getenv('MSSQL_USER', '')m_pwd  = os.getenv('MSSQL_PASSWORD', '')m_enc  = os.getenv('MSSQL_ENCRYPT', 'yes')m_tsc  = os.getenv('MSSQL_TRUST_CERT', 'yes')m_tds  = os.getenv('MSSQL_TDS_VERSION', '7.4')conn_str = (    f"DRIVER={{FreeTDS}};SERVER={m_host};PORT={m_port};"    f"DATABASE={m_db};UID={m_user};PWD={m_pwd};"    f"TDS_Version={m_tds};Encrypt={m_enc};TrustServerCertificate={m_tsc};")print('ODBC connect string (safe):', conn_str.replace(m_pwd, '***'))try:    with pyodbc.connect(conn_str, timeout=5) as cx:        cur = cx.cursor()        cur.execute('SELECT @@VERSION')        row = cur.fetchone()        print('âœ… SQL Server connected. Version snippet:')        print(str(row[0]).split('\n')[0])        cur.execute('SELECT 1 AS one')        print('Test query =>', cur.fetchone())except Exception as e:    print('âŒ SQL Server connection failed:', e)

## Test MariaDB / MySQL (PyMySQL)

In [ ]:
import os, pymysqlhost = os.getenv('MARIADB_HOST', 'host.docker.internal')port = int(os.getenv('MARIADB_PORT', '3306'))db   = os.getenv('MARIADB_DB', '')user = os.getenv('MARIADB_USER', '')pwd  = os.getenv('MARIADB_PASSWORD', '')print(f'Connecting to MariaDB at {host}:{port} / {db} as {user}')try:    cx = pymysql.connect(host=host, port=port, user=user, password=pwd, database=db)    with cx.cursor() as cur:        cur.execute('SELECT VERSION()')        print('âœ… MariaDB connected. Version:', cur.fetchone()[0])        cur.execute('SELECT 1 AS one')        print('Test query =>', cur.fetchone())    cx.close()except Exception as e:    print('âŒ MariaDB connection failed:', e)

## Test with pandas.read_sql()

In [ ]:
import pandas as pdimport pyodbc, pymysqlfrom contextlib import closingimport os# SQL Server via pyodbctry:    conn_str = (        f"DRIVER={{FreeTDS}};SERVER={os.getenv('MSSQL_HOST','host.docker.internal')};PORT={os.getenv('MSSQL_PORT','1433')};"        f"DATABASE={os.getenv('MSSQL_DB','')};UID={os.getenv('MSSQL_USER','')};PWD={os.getenv('MSSQL_PASSWORD','')};"        f"TDS_Version={os.getenv('MSSQL_TDS_VERSION','7.4')};Encrypt={os.getenv('MSSQL_ENCRYPT','yes')};TrustServerCertificate={os.getenv('MSSQL_TRUST_CERT','yes')};"    )    with closing(pyodbc.connect(conn_str, timeout=5)) as cx:        df = pd.read_sql('SELECT 1 AS one, 2 AS two', cx)        print('âœ… pandas from SQL Server:')        display(df)except Exception as e:    print('âŒ pandas+pyodbc failed:', e)# MariaDB via PyMySQLtry:    cx = pymysql.connect(      host=os.getenv('MARIADB_HOST','host.docker.internal'),      port=int(os.getenv('MARIADB_PORT','3306')),      user=os.getenv('MARIADB_USER',''),      password=os.getenv('MARIADB_PASSWORD',''),      database=os.getenv('MARIADB_DB','')    )    df2 = pd.read_sql('SELECT 1 AS one, 3 AS three', cx)    cx.close()    print('âœ… pandas from MariaDB:')    display(df2)except Exception as e:    print('âŒ pandas+PyMySQL failed:', e)